In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("quiz_07.ipynb")

# Quiz 7

## Instructions

1. ***Do not change any file names or relative locations***
1. After you answer a question, you should run its public tests.
1. After you answer every question, you should:
    1. Restart your kernel and clear all output
    1. Run up to the last cell
    1. Save your notebook
    1. Run the last cell to create a .zip file for Gradescope
    1. Upload this .zip file to Gradescope
    1. ***Make sure your Gradescope autograder results match your local autograder results***
1. This quiz has public and hidden tests:
    1. Public tests check your answers for correct types and shapes but may not completely check your answers
    1. Hidden tests completely check your answers but will not be available until after the due date
    1. *If all tests were public, you could quickly reverse-engineer incorrect answers that trick the autograder*
1. You may ask technical questions on Canvas Discussions, but this quiz is an individual effort.

## Packages and Settings

***The following code cell imports all the packages you need for this quiz.***
***Do not import any other packages.***

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.optimize as sco

## Data

I used the following code to download and save the data you need for this quiz.
***You should review the following code but not run it.***

In [3]:
# import yfinance as yf

# (
#     yf.download(tickers=['MSFT', 'AAPL', 'TSLA', 'AMZN', 'NVDA', 'GOOG'])
#     .assign(Date = lambda x: x.index.tz_localize(None))
#     .set_index('Date')
#     .rename_axis(columns=['Variable', 'Ticker'])
#     .to_csv('yahoo.csv')
# )

## Questions

### Question 1

Complete the following calculations:

1. Read the daily price data for the MATANA stocks saved in file `yahoo.csv` and assign them to data frame `yahoo`
1. Calculate the maximum Sharpe Ratio portfolio weights assign them to array `weights` 
    1. Use the daily returns on the MATANA stocks from 2011 through 2022 (inclusive)
    1. Use a target or benchmark return of 0 (instead of the risk-free rate)
    1. Calculate long-only portfolio weights, but do not allow any portfolio weight to exceed 0.3
1. Calculate rolling Sharpe Ratios for the maximum Sharpe Ratio portfolio and assign them to series `sr_rolling`
    1. Use 252-trading day windows
    1. Require at least 200 trading days to calculate each Sharpe Ratio
    
This question has 6 hidden tests worth 5 points each.

_Points:_ 100

In [6]:
#1
yahoo = pd.read_csv('yahoo.csv',header=[0, 1],index_col=0, parse_dates=True)
returns = yahoo.loc['2010/12/31':'2022/12/31', 'Adj Close'].pct_change().dropna()

#2
def port_sharpe(x, r, tgt, ppy):
    rp = r.dot(x)
    er = rp.sub(tgt)
    return -1 * np.sqrt(ppy) * er.mean() / er.std()

res_sharpe_1 = sco.minimize(
    fun=port_sharpe,
    x0=np.ones(returns.shape[1]) / returns.shape[1],
    args=(returns, 0, 252),
    bounds=[(0,0.3) for _ in returns],
    constraints=(
        {'type': 'eq', 'fun': lambda x: x.sum() - 1} 
    )
)
weights = res_sharpe_1['x']

def print_port_res(w, r, title, ppy=252, tgt=None):
    width = len(title)
    rp = r.dot(w)
    mu = ppy * rp.mean()
    sigma = np.sqrt(ppy) * rp.std()
    if tgt is not None:
        er = rp.sub(tgt)
        sr = np.sqrt(ppy) * er.mean() / er.std()
    else:
        sr = None
    
    return print(
        title,
        '=' * width,
        '',
        'Performance',
        '-' * width,
        'Return:'.ljust(width - 6) + f'{mu:0.4f}',
        'Volatility:'.ljust(width - 6) + f'{sigma:0.4f}',
        'Sharpe Ratio:'.ljust(width - 6) + f'{sr:0.4f}\n' if sr is not None else '',
        'Weights', 
        '-' * width, 
        '\n'.join([f'{_r}:'.ljust(width - 6) + f'{_w:0.4f}' for _r, _w in zip(r.columns, w)]),
        sep='\n',
    )
print_port_res(w=res_sharpe_1['x'], r=returns,title = ' ')

 
=

Performance
-
Return:0.3423
Volatility:0.2913

Weights
-
AAPL:0.2801
AMZN:0.0425
GOOG:0.0000
MSFT:0.2371
NVDA:0.2062
TSLA:0.2340


In [7]:
yahoo.shape == (10671, 36)

True

In [8]:
np.allclose(yahoo['Adj Close'].sum().values, np.array([177461.84397135, 206208.61394934, 180642.82432055, 393585.96007783, 187086.25384754, 197001.72133398]))

True

In [9]:
weights.shape == (6,)

True

In [10]:
((0 <= weights) & (weights <= 0.3)).all()

True

In [11]:
np.allclose(weights.sum(), 1)

True

In [12]:
#3
def sharpe(ri, rf=0, ann_fac=np.sqrt(252)):
    ri_rf = ri.sub(rf).dropna()
    return ann_fac * ri_rf.mean() / ri_rf.std()

port = returns.dot(weights)
print(port)
sr_rolling = port.loc['2011':].rolling(252,200).apply(sharpe)

Date
2011-01-03    0.013216
2011-01-04    0.002364
2011-01-05    0.019312
2011-01-06    0.044061
2011-01-07    0.008894
                ...   
2022-12-23   -0.005400
2022-12-27   -0.048158
2022-12-28   -0.005152
2022-12-29    0.042956
2022-12-30    0.002198
Length: 3020, dtype: float64


In [13]:
sr_rolling.shape == (3020,)

True

In [14]:
sr_rolling.notna().sum() == 2821

True

In [15]:
sr_rolling.sum() > 0

True

In [16]:
sr_rolling.sum() > 500

True

In [17]:
grader.check("q1")

q1 results: All test cases passed!
q1 - 1 message: yahoo has the right shape (+10 pts)
q1 - 2 message: yahoo has the right Adj Close column sums (+10 pts)
q1 - 3 message: weights has the right shape (+10 pts)
q1 - 4 message: weights values fall between 0 and 0.3 (+10 pts)
q1 - 5 message: weights sum to 1 (+10 pts)
q1 - 6 message: sr_rolling has the right shape (+5 pts)
q1 - 7 message: sr_rolling has the right number of non-missing values (+5 pts)
q1 - 8 message: sr_rolling has a positive sum (+5 pts)
q1 - 9 message: sr_rolling has a sum > 500 (+5 pts)

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [34]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False, run_tests=True)

Running your submission against local test cases...



Your submission received the following results when run against available test cases:

    q1 results: All test cases passed!
    q1 - 1 message: yahoo has the right shape (+10 pts)
    q1 - 2 message: yahoo has the right Adj Close column sums (+10 pts)
    q1 - 3 message: weights has the right shape (+10 pts)
    q1 - 4 message: weights values fall between 0 and 0.3 (+10 pts)
    q1 - 5 message: weights sum to 1 (+10 pts)
    q1 - 6 message: sr_rolling has the right shape (+5 pts)
    q1 - 7 message: sr_rolling has the right number of non-missing values (+5 pts)
    q1 - 8 message: sr_rolling has a positive sum (+5 pts)
    q1 - 9 message: sr_rolling has a sum > 500 (+5 pts)
